In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_json('/content/drive/MyDrive/datasets/case.json')
df.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 08:04:08 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 11:43:23 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 09:04:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
3,2021-08-25 05:02:55 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
4,2021-09-05 08:03:28 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."


In [ ]:
df['EventName'].unique()

array(['DynamicPrice_Result', 'CurateOffer_Result'], dtype=object)

In [ ]:
df.shape

(37, 3)

Curated Offer Options

In [ ]:
curated_df = df[df['EventName'] == 'CurateOffer_Result']
curated_df.head()

,EnqueuedTimeUtc,EventName,Payload
3,2021-08-25 05:02:55 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
13,2021-08-25 09:03:29 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""c99a..."
15,2021-08-25 05:02:18 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""d668..."
17,2021-08-25 09:02:29 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""0a06..."
35,2021-08-25 09:03:14 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""135d..."


In [ ]:
def create_curatedofferoption_df(row):
  enqueued_time = row["EnqueuedTimeUtc"]
  payload_row = row['Payload']
  payload_row = json.loads(payload_row)
  result = []
  for val in payload_row:
    curation_provider = f"\"{val['curationProvider']}\""
    offer_id = f"\"{val['offerId']}\""
    dealer_id = f"\"{val['dealerId']}\""
    for option in val['options']:
       offer_options = {
           "CurationProvider": curation_provider,
           "OfferId": offer_id,
           "DealerId": dealer_id,
           "UniqueOptionId" : f"\"{option['uniqueOptionId']}\"",
           "Option_id" : f"\"{option['optionId']}\"",
           "IsMobileDealer" : f"\"{option['isMobileDealer']}\"",
           "isOpen" : f"\"{option['isOpen']}\"",
           "Eta" : f"\"{option['eta']}\"",
           "ChamaScore" : f"\"{option['chamaScore']}\"",
           "ProductBrand" : f"\"{option['productBrand']}\"",
           "isWinner" : f"\"{option['isWinner']}\"",
           "MinimumPrice" : f"\"{option['minimumPrice']}\"",
           "MaximumPrice" : f"\"{option['maximumPrice']}\"",
           "DynamicPrice" : f"\"{option['dynamicPrice']}\"",
           "FinalPrice" : f"\"{option['finalPrice']}\""
       }
       if "defeatPrimaryReason" in option:
         offer_options["DefeatPrimaryReason"] = f"\"{option['defeatPrimaryReason']}\""
         offer_options["DefeatReasons"] = f"\"{option['defeatReasons']}\""
       else:
         offer_options["DefeatPrimaryReason"] = "\"\""
         offer_options["DefeatReasons"] = "\"\""
       offer_options["EnqueuedTime"] = enqueued_time
       result.append(offer_options)
  return result

flatten = [x for xs in curated_df.apply(create_curatedofferoption_df, axis=1) for x in xs]
curated_offer_payload = pd.DataFrame(flatten)
curated_offer_payload.head()

,CurationProvider,OfferId,DealerId,UniqueOptionId,Option_id,IsMobileDealer,isOpen,Eta,ChamaScore,ProductBrand,isWinner,MinimumPrice,MaximumPrice,DynamicPrice,FinalPrice,DefeatPrimaryReason,DefeatReasons,EnqueuedTime
0,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""b0e296a9-0590-f0e0-8211-243a2ededb12""","""6517 || dd839e4c-9f84-45eb-9cb2-9069fecf70f2""","""True""","""True""","""1:00""","""8.0""","""ULTRAGAZ""","""True""","""90.0""","""180.0""","""91.9""","""91.9""","""""","""""",2021-08-25 05:02:55 UTC
1,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""6517""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""","""6517 || 6517""","""False""","""False""","""0:01""","""8.0""","""ULTRAGAZ""","""False""","""90.0""","""180.0""","""91.9""","""91.9""","""Closed""","""['Closed', 'HasDriverInOffer']""",2021-08-25 05:02:55 UTC
2,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""","""9047 || 9047 || ULTRAGAZ""","""False""","""False""","""1:00""","""9.0""","""ULTRAGAZ""","""False""","""99.0""","""198.0""","""99.95""","""99.95""","""Closed""","""['Closed', 'HigherPrice', 'HasDriverInOffer']""",2021-08-25 05:02:55 UTC
3,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""","""9047 || 9047 || CONSIGAZ""","""False""","""False""","""1:00""","""9.0""","""CONSIGAZ""","""False""","""89.99""","""179.98""","""91.89""","""91.89""","""Closed""","""['Closed', 'HigherPrice', 'HigherETA']""",2021-08-25 05:02:55 UTC
4,"""ByPrice""","""149f0e53-ff85-425f-a01a-8710f06704ea""","""9047""","""577e4bbd-f49d-ac23-56a6-e70072a05229""","""9047 || 9047 || LIQUIGAS""","""False""","""False""","""1:00""","""9.0""","""LIQUIGAS""","""False""","""92.0""","""184.0""","""93.9""","""93.9""","""Closed""","""['Closed', 'HigherPrice', 'HigherETA']""",2021-08-25 05:02:55 UTC


## DynamicPriceRange

In [ ]:
dynamicprice_df = df[df['EventName'] == 'DynamicPrice_Result']
dynamicprice_df.head()

,EnqueuedTimeUtc,EventName,Payload
0,2021-09-05 08:04:08 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
1,2021-08-18 11:43:23 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."
2,2021-09-05 09:04:04 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
4,2021-09-05 08:03:28 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPriceRange"",""offerId""..."
5,2021-08-18 11:44:00 UTC,DynamicPrice_Result,"{""provider"":""ApplyDynamicPricePerOption"",""offe..."


In [ ]:
def create_dynamicprice_df(row):
  enqueued_time = row["EnqueuedTimeUtc"]
  payload_row = row['Payload']
  payload_row = json.loads(payload_row)
  result = []
  if payload_row["provider"] == "ApplyDynamicPriceRange":
    provider = f"\"{payload_row['provider']}\""
    offer_id = f"\"{payload_row['offerId']}\""
    algorithm_output = payload_row["algorithmOutput"]
    offer_options = {
          "Provider": provider,
          "OfferId": offer_id,
          "MinGlobal":f"\"{['min_global']}\"",
          "MinRecommended":f"\"{algorithm_output['min_recommended']}\"",
          "MaxRecommended":f"\"{algorithm_output['max_recommended']}\"",
          "DifferenceMinRecommendMinTheory":f"\"{algorithm_output['differenceMinRecommendMinTheory']}\"",
          "EnqueuedTimeSP": enqueued_time
      }
    result.append(offer_options)
  return result

flatten = [x for xs in dynamicprice_df.apply(create_dynamicprice_df, axis=1) for x in xs]
dynamicprice_payload = pd.DataFrame(flatten)
dynamicprice_payload


,Provider,OfferId,MinGlobal,MinRecommended,MaxRecommended,DifferenceMinRecommendMinTheory,EnqueuedTimeSP
0,"""ApplyDynamicPriceRange""","""a6611d55-9624-4381-8cdd-323ee3689241""","""['min_global']""","""87.2""","""97.65""","""2.2""",2021-09-05 08:04:08 UTC
1,"""ApplyDynamicPriceRange""","""b8c636fa-8241-47dc-ac40-bdf438a04d9c""","""['min_global']""","""87.2""","""97.65""","""2.2""",2021-09-05 09:04:04 UTC
2,"""ApplyDynamicPriceRange""","""3d32f7fb-396d-4d3f-b673-dea1f7dc41b7""","""['min_global']""","""87.2""","""97.65""","""2.2""",2021-09-05 08:03:28 UTC
3,"""ApplyDynamicPriceRange""","""329194f3-95a4-45ef-b3d0-2796f74ce2a0""","""['min_global']""","""87.2""","""97.65""","""2.2""",2021-09-05 08:03:40 UTC
4,"""ApplyDynamicPriceRange""","""fdcfde5c-113d-4a59-9ae0-8bc31e2943d8""","""['min_global']""","""89.25""","""99.95""","""1.9""",2021-09-05 04:03:08 UTC
5,"""ApplyDynamicPriceRange""","""27bbc4fa-2388-4780-b66c-92a51397d191""","""['min_global']""","""89.25""","""99.95""","""1.9""",2021-09-05 09:18:04 UTC
6,"""ApplyDynamicPriceRange""","""baffc30b-7642-45fe-a2ce-da31a71732ae""","""['min_global']""","""87.2""","""97.65""","""2.2""",2021-09-05 08:04:16 UTC
7,"""ApplyDynamicPriceRange""","""b5982abd-f602-47ac-b45a-bb43bf993d46""","""['min_global']""","""87.2""","""97.65""","""2.2""",2021-09-05 09:03:24 UTC
8,"""ApplyDynamicPriceRange""","""f6643886-4a0f-45ae-ae32-ee95c72ee94a""","""['min_global']""","""89.25""","""99.95""","""1.9""",2021-09-05 04:03:52 UTC
9,"""ApplyDynamicPriceRange""","""16a2d492-b1c3-40ec-970f-b8704d8db96f""","""['min_global']""","""87.2""","""97.65""","""2.2""",2021-09-05 08:03:00 UTC


In [ ]:

def dynamic_price_option(row):
    """Transform the JSON payload into a dynamic price option dataframe."""
    enqueued_time = row["EnqueuedTimeUtc"]
    payload_row = row["Payload"]
    payload_row = json.loads(payload_row)
    result = []
    if payload_row["provider"] == "ApplyDynamicPricePerOption":
        provider = f"\"{payload_row['provider']}\""
        offer_id = f"\"{payload_row['offerId']}\""
        for unique_option in payload_row["algorithmOutput"]:
            price_option = {
                "Provider": provider,
                "OfferId": offer_id,
                "UniqueOptionId": f"\"{unique_option['uniqueOptionId']}\"",
                "BestPrice": unique_option["bestPrice"],
                "EnqueuedTimeSP": enqueued_time
            }
            result.append(price_option)
    return result


lst = dynamicprice_df.apply(dynamic_price_option, axis=1)
flatten = [x for xs in lst for x in xs]
dynamic_price_option_payload = pd.DataFrame(flatten)
dynamic_price_option_payload.head()

,Provider,OfferId,UniqueOptionId,BestPrice,EnqueuedTimeSP
0,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""b0e296a9-0590-f0e0-8211-243a2ededb12""",92.45,2021-08-18 11:43:23 UTC
1,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""d6562c24-0b37-5fb4-8275-65b7b8b47b87""",92.45,2021-08-18 11:43:23 UTC
2,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""8d0f9262-f543-d0c8-a869-33985ae3ecda""",92.45,2021-08-18 11:43:23 UTC
3,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""151e59ac-761a-96f5-d2b9-882037a9fd28""",94.60,2021-08-18 11:43:23 UTC
4,"""ApplyDynamicPricePerOption""","""56e0702c-0218-4626-8d3d-ae9d54b4503b""","""3cd346f4-d297-7568-2e50-d43a8e2fd0a9""",94.60,2021-08-18 11:43:23 UTC
